In [11]:
class User:
  def __init__(self, user_id, name):
    self.id = user_id
    self.name = name
    self.friends = []
    self.ratings = {}
    self.centrality_score = None  # Placeholder for network centrality


In [12]:
# Define movie class (optional)
class Movie:
  def __init__(self, movie_id, title):
    self.id = movie_id
    self.title = title

# Network and Centrality Score Calculation (using NetworkX)

In [13]:
import networkx as nx

def calculate_centrality_scores(users):
  G = nx.Graph()
  G.add_edges_from([(u.id, friend.id) for u in users.values() for friend in u.friends])
  for user_id, user in users.items():
    user.centrality_score = nx.degree_centrality(G)[user_id]  # Example: Degree Centrality


In [14]:
# Function to calculate rating similarity (using Pearson correlation)
from scipy.stats import pearsonr

def calculate_similarity(user1_ratings, user2_ratings):
  common_movies = set(user1_ratings.keys()) & set(user2_ratings.keys())
  if len(common_movies) < 2:  # Check for at least 2 common movies
    return 0
  ratings1 = [user1_ratings[movie] for movie in common_movies]
  ratings2 = [user2_ratings[movie] for movie in common_movies]
  _, correlation = pearsonr(ratings1, ratings2)
  return correlation



In [15]:
# Define a simple Bayesian network class (assuming binary ratings)
class BayesianNetwork:
  def __init__(self, users):
    self.users = users
    # Pre-calculate conditional probabilities (can be learned from data)
    self.p_user_rating_given_friend_rating = {
      (user_id, rating): {friend_id: 0.7 for friend_id in users}  # Example: Prior of 0.7
      for user_id in users
      for rating in (0, 1)
    }

  def update_conditional_probability(self, user1, user2, rating1, rating2):
    # Update conditional probability based on observed ratings (example)
    self.p_user_rating_given_friend_rating[user1.id][user2.id] = 0.8 if rating1 == rating2 else 0.2

  def predict_rating(self, user, movie, friend):
    friend_rating = friend.ratings.get(movie, None)
    if friend_rating is None:
      return None  # Friend hasn't rated the movie
    prior = self.p_user_rating_given_friend_rating[user.id][friend.id]
    return prior * friend_rating


In [16]:
# Function to find similar users based on a combination of factors
def get_similar_users(user, network, threshold=0.2):
  similar_users = []
  for friend in user.friends:
    if not friend.ratings:  # Skip friends with no ratings
      continue
    similarity = 0.7 * calculate_similarity(user.ratings, friend.ratings)  # Weight rating similarity
    similarity += 0.3 * friend.centrality_score  # Weight centrality score (adjustable)
    if similarity >= threshold:
      similar_users.append(friend)
  return similar_users


In [17]:
# Function to recommend movies
def recommend_movies(user, network):
  recommendations = {}
  for friend in get_similar_users(user, network):
    for movie, rating in friend.ratings.items():
      if movie not in user.ratings:
        predicted_rating = network.predict_rating(user, movie, friend)
        if predicted_rating is not None:
          recommendations[movie] = recommendations.get(movie, 0) + predicted_rating
  # Sort recommendations based on score
  sorted_recommendations = sorted(recommendations.items(), key=lambda x: x[1], reverse=True)
  return sorted_recommendations[:5]

In [18]:
users = {
  "U1": User("U1", "Alice"),
  "U2": User("U2", "Bob"),
  "U3": User("U3", "Charlie"),
  "U4": User("U4", "Diana"),
  "U5": User("U5", "Emily"),
}

movies = {
  "M1": "The Godfather",
  "M2": "The Shawshank Redemption",
  "M3": "The Dark Knight",
}

# Add friends (based on social network data)
users["U1"].friends.append(users["U2"])
users["U1"].friends.append(users["U3"])
users["U2"].friends.append(users["U4"])
users["U3"].friends.append(users["U5"])

# Add movie ratings (based on sample data)
users["U1"].ratings[movies["M1"]] = 4
users["U1"].ratings[movies["M2"]] = 5
users["U2"].ratings[movies["M1"]] = 3
users["U2"].ratings[movies["M3"]] = 4
users["U3"].ratings[movies["M2"]] = 5
users["U4"].ratings[movies["M1"]] = 5
users["U4"].ratings[movies["M3"]] = 2


In [19]:
# Example usage
calculate_centrality_scores(users)
network = BayesianNetwork(users)

user = users["U1"]
recommendations = recommend_movies(user, network)

print(f"Movie recommendations for {user.name}:")
for movie_id, score in recommendations:
  print(f"- {movies[movie_id]} (score: {score:.2f})")


Movie recommendations for Alice:
